In [3]:
# Voting Improve

In [4]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dropout,Flatten,Dense,Activation,concatenate
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
X_train=pickle.load(open("X_train","rb"))
X_test=pickle.load(open("X_test","rb"))
y_train=pickle.load(open("y_train","rb"))
y_test=pickle.load(open("y_test","rb"))

In [6]:
def create_model():
    inputl=Input(shape=(128,128,1))

    x=Conv2D(96,(3,3),padding="same")(inputl)
    x=Activation("relu")(x)
    x=MaxPooling2D(pool_size=(2,2))(x)
    x=Dropout(0.3)(x)

    x=Conv2D(72,(3,3),padding="same")(x)
    x=Activation("relu")(x)
    x=MaxPooling2D(pool_size=(2,2))(x)
    x=Dropout(0.3)(x)

    x1=Conv2D(72,(3,3),padding="same")(x)
    x1=Activation("relu")(x1)
    x1=MaxPooling2D(pool_size=(2,2))(x1)
    x1=Dropout(0.3)(x1)
    x2=Conv2D(32,(5,5),padding="same")(x)
    x2=Activation("relu")(x2)
    x2=MaxPooling2D(pool_size=(2,2))(x2)
    x2=Dropout(.3)(x2)
    x3=Conv2D(32,(7,7),padding="same")(x)
    x3=Activation("relu")(x3)
    x3=MaxPooling2D(pool_size=(2,2))(x3)
    x3=Dropout(0.3)(x3)

    x=concatenate([x1,x2,x3],axis=3)

    x=Flatten()(x)

    x=Dense(84, activation="relu",kernel_regularizer=regularizers.l2(l=0.001),bias_regularizer=regularizers.l2(l=0.001))(x)
    x=Dropout(0.3)(x)
    x=Dense(128, activation="relu",kernel_regularizer=regularizers.l2(l=0.001),bias_regularizer=regularizers.l2(l=0.001))(x)
    x=Dropout(0.5)(x)
    x=Dense(128, activation="relu",kernel_regularizer=regularizers.l2(l=0.001),bias_regularizer=regularizers.l2(l=0.001))(x)
    x=Dropout(0.3)(x)
    outputl=Dense(1,activation="sigmoid")(x)
    model=Model(inputs=inputl,outputs=outputl)
    opt= tf.keras.optimizers.Adam(learning_rate=0.0002)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy",tf.keras.metrics.AUC(name='auc'),tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')])
    return model
model_f=create_model()

In [7]:
predict_result_train=[]
predict_result_test=[]

In [8]:
model_f.load_weights('difker_3020e.h5')
predict_result_train.append(model_f.predict(X_train))
predict_result_test.append(model_f.predict(X_test))

In [9]:
def create_model():
    inputl=Input(shape=(128,128,1))

    x=Conv2D(96,(3,3),padding="same")(inputl)
    x=Activation("relu")(x)
    x=MaxPooling2D(pool_size=(2,2))(x)

    x=Conv2D(72,(3,3),padding="same")(x)
    x=Activation("relu")(x)
    x=MaxPooling2D(pool_size=(2,2))(x)

    x1=Conv2D(72,(3,3),padding="same")(x)
    x1=Activation("relu")(x1)
    x1=MaxPooling2D(pool_size=(2,2))(x1)

    x2=Conv2D(32,(5,5),padding="same")(x)
    x2=Activation("relu")(x2)
    x2=MaxPooling2D(pool_size=(2,2))(x2)

    x3=Conv2D(32,(7,7),padding="same")(x)
    x3=Activation("relu")(x3)
    x3=MaxPooling2D(pool_size=(2,2))(x3)

    x=concatenate([x1,x2,x3],axis=3)

    x=Flatten()(x)

    x=Dense(84, activation="relu",kernel_regularizer=regularizers.l2(l=0.001),bias_regularizer=regularizers.l2(l=0.001))(x)
    x=Dropout(0.3)(x)
    x=Dense(128, activation="relu",kernel_regularizer=regularizers.l2(l=0.001),bias_regularizer=regularizers.l2(l=0.001))(x)
    x=Dropout(0.5)(x)
    x=Dense(128, activation="relu",kernel_regularizer=regularizers.l2(l=0.001),bias_regularizer=regularizers.l2(l=0.001))(x)
    x=Dropout(0.3)(x)
    outputl=Dense(1,activation="sigmoid")(x)
    model=Model(inputs=inputl,outputs=outputl)
    opt= tf.keras.optimizers.Adam(learning_rate=0.0002)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy",tf.keras.metrics.AUC(name='auc'),tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')])
    return model

In [10]:
for i in range(10):
    model=create_model()
    model.load_weights('model'+str(i)+'.h5')
    predict_result_train.append(model.predict(X_train))
    predict_result_test.append(model.predict(X_test))

In [11]:
predict_result_train=np.array(predict_result_train)
predict_result_test=np.array(predict_result_test)

In [12]:
predict_result_train.shape

(11, 36228, 1)

In [13]:
predict_result_train=predict_result_train.reshape(11,36228)

In [14]:
predict_result_test.shape

(11, 9057, 1)

In [15]:
predict_result_test=predict_result_test.reshape(11,9057)

In [16]:
predict_result_train=np.transpose(predict_result_train)
predict_result_train.shape

(36228, 11)

In [17]:
predict_result_test=np.transpose(predict_result_test)
predict_result_test.shape

(9057, 11)

In [18]:
pickle_out=open("predict_result_train","wb")
pickle.dump(predict_result_train,pickle_out)
pickle_out.close()

pickle_out=open("predict_result_test","wb")
pickle.dump(predict_result_test,pickle_out)
pickle_out.close()

In [34]:
def vote_model():
    inputl=Input(shape=(11,))
    x=Dense(128, activation="relu",kernel_regularizer=regularizers.l2(l=0.001),bias_regularizer=regularizers.l2(l=0.001))(inputl)
    x=Dropout(0.5)(x)
    outputl=Dense(1,activation="sigmoid")(inputl)
    model=Model(inputs=inputl,outputs=outputl)
    opt= tf.keras.optimizers.Adam(learning_rate=0.0002)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy",tf.keras.metrics.AUC(name='auc'),tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')])
    return model

In [35]:
vote=vote_model()

In [36]:
vote.fit(predict_result_train,y_train,epochs=10,batch_size=10,validation_data=(predict_result_test,y_test))

Epoch 1/10
3623/3623 [==============================] - 9s 2ms/step - loss: 0.9004 - accuracy: 0.4006 - auc: 0.4245 - precision: 0.4139 - recall: 0.5050 - val_loss: 0.6303 - val_accuracy: 0.6804 - val_auc: 0.8151 - val_precision: 0.6158 - val_recall: 0.9684
Epoch 2/10
3623/3623 [==============================] - 9s 2ms/step - loss: 0.4715 - accuracy: 0.7906 - auc: 0.9643 - precision: 0.7051 - recall: 0.9923 - val_loss: 0.4859 - val_accuracy: 0.7868 - val_auc: 0.9050 - val_precision: 0.7201 - val_recall: 0.9420
Epoch 3/10
3623/3623 [==============================] - 8s 2ms/step - loss: 0.3238 - accuracy: 0.8990 - auc: 0.9816 - precision: 0.8413 - recall: 0.9813 - val_loss: 0.4178 - val_accuracy: 0.8237 - val_auc: 0.9082 - val_precision: 0.7838 - val_recall: 0.8966
Epoch 4/10
3623/3623 [==============================] - 8s 2ms/step - loss: 0.2516 - accuracy: 0.9252 - auc: 0.9831 - precision: 0.8907 - recall: 0.9678 - val_loss: 0.3943 - val_accuracy: 0.8326 - val_auc: 0.9092 - val_precisi

It seems that the neural network still does not change much.
It may be the problem of the 10 underfitting models.